In [27]:
#This file takes the SLA observations from both AVISO and Scott's interpolations and converts it to ADT; this is step two in the preprocessing for the eddy observations
import xarray as xr

In [28]:
#FILL IN INFORMATION HERE
from datetime import datetime
from datetime import timedelta, date

filepath="rawdata/" #path to netcdf file with SSH

tmax=30 #total number of days
savepath="eddies/" #path to save the eddies
filenamepath="filenames.npy"
slice_=False
longmin=360-62 #minimum longitude
longmax=360-50 #maximum longitude
latmin=25 #minimum latitude
latmax=52 #maximum latitude
def findfilename(path,t):
    return path+str(t)+".nc"
startdate=datetime(2021,11,15)
filepath1="scottcopy/rawdata/"  #This is the filepath to the AVISO data
filepath2="/home/mavida/dat1/miniconda3/eddytracking/testsonscottsdata/rawdata/" #This is the filepath to Scott's interpolations
writepath1="scottcopy/rawdata2/" #This is where the new AVISO data should go
writepath2="testsonscottsdata/rawdata2/" #This is where Scott's new data shou go
mdtfilepath="scottcopy/mdt.nc" #put the path to where the MDT is stored
mdt=xr.open_dataset(mdtfilepath)

In [41]:
#For each day in the specified range, it adds ADT to each file
for t in range(tmax):
    filename1=findfilename(filepath1,t)
    filename2=findfilename(filepath2,t)
    writename1=findfilename(writepath1,t)
    writename2=findfilename(writepath2,t)
    for filename,add,writename in [[filename1,0,writename1],[filename2,360,writename2]]:
        dataset=xr.open_dataset(filename)
        interpolatedmdt=mdt.isel(time=0).interp(longitude=dataset["longitude"]-add,latitude=dataset["latitude"])["mdt"]
        dataset=dataset.assign(adt=dataset["sla"]+interpolatedmdt)
        dataset["adt"].attrs["units"] = "m"
        dataset["adt"]
        dataset.to_netcdf(path=writename, mode='w')